In [1]:
print("start")
import os
import sys
%cd ..
from utils import get_data,ColumnsPreprocessing,get_specific_df,ColumnsTrainKfold,seedEverything,grid_parameters_name,grid_parameters,TrainOneFold,get_cv_split
from sklearn.model_selection import GridSearchCV
from stg_fs import get_stg_class,get_SelectFdr_class,get_mrmr_class,get_reliefF_class,get_RFE_SVM_class,get_FWDT_class,get_ensemble_class, get_ensemble_class_new ,get_stg_class_new,get_FWDT_class_new
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
import numpy as np
import time
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import clear_output
from joblib import Parallel, delayed
import pickle
from calculate_metric_score import get_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
# from main import *
from tqdm.auto import tqdm
from IPython.display import clear_output
import torch

start
/sise/liorrk-group/OrDanOfir/lior.exp.4


In [2]:
def run_GridSearchCV(train_features,y,fun,name,cv=3):
    if os.path.exists(f'output/Preprocessing/{name}.pickle'):
        with open(f'output/Preprocessing/{name}.pickle', 'rb') as handle:
            results = pickle.load(handle)
        return results
    results = {}
    for n,params in tqdm(zip(grid_parameters_name,grid_parameters)):
        clf = params['clf'][0]
        params2 = params.copy()
        params2.pop('clf')
        estimators = [("SelectKBest",SelectKBest(fun,k=100)), ('clf', clf)]
        pipe = Pipeline(estimators)
        grid = GridSearchCV(pipe, param_grid=params2, cv=cv,n_jobs=-1)
        _=grid.fit(train_features,y)
        results[n] = {}
        for row in grid.best_params_:
            results[n][row.split("__")[-1]] = grid.best_params_[row]
    with open(f'output/Preprocessing/{name}.pickle', 'wb') as handle:
        pickle.dump(results, handle, protocol=4)
    return results

In [3]:
# s = time.time()
# fun = Filtering[Filtering_Algorithm](datasets=name,out_path=f'temp/{Filtering_Algorithm}')
# results = run_GridSearchCV(train_features,y,fun,name,cv=3)
# print(time.time()-s)

In [4]:
from utils import pca
from imblearn.over_sampling import SMOTE

def train_kfold(get_stg_fun,
                            Filtering_Algorithm,
                            k,
                            datsets_num,
                            f_outpot,
                            PP,
                            end,
                            Learning_algorithm,
                            knn_args = {"n_neighbors": 5},
                            rf_args = {"n_estimators":100},
                            lr_args = {"C":1e5},
                            SVC_args = {"C":1,"probability":True},
                            NB_args = {"alpha":1}):

    knn = KNeighborsClassifier(**knn_args)
    rf = RandomForestClassifier(**rf_args)
    lr = LogisticRegression(**lr_args)
    svc = SVC(**SVC_args)
    BN = BernoulliNB(**NB_args)
    # models = {"knn":knn,"rf":rf,"lr":lr,"SVC":svc,"BN":BN}
    models = {Learning_algorithm : eval(Learning_algorithm if Learning_algorithm!="SVC" else 'svc')}
    history = {}
    df,name = get_specific_df(datsets_num)
    skf,n_splits,name_cv = get_cv_split(df)
    train_features,y = PP.transform(df)
    start = time.time()
    score_gates = get_stg_fun(train_features.values,y.values,k=k)
    topk = score_gates.argsort()[::-1][0:k]
    col_name = np.array(df.drop('target',axis=1).columns)[topk]
    score_gates = score_gates[topk]
    history[f"time_gates"] = end
    history[f"score_gates"] = score_gates
    history[f"col_name"] = col_name
    
    score_dict = {}
    run_only_cv = False if "Folds" in name_cv else True
    split_fun = skf.split(train_features, y) if "Folds" in name_cv else skf.split(train_features)
    train_features = train_features.values
    y = y.values
    for clf in models:
        history[clf] = {}
        history[clf]['score'] = {}
        history[clf]['y_score'] = []
        history[clf]['y_val'] = []
        history[clf]['index_val'] = []
        X = SelectKBest(get_stg_fun,k=k).fit_transform(train_features, y)
    for fold,(train_index, test_index) in enumerate(tqdm(split_fun)):
        X_train, X_test = train_features[train_index].copy(), train_features[test_index].copy()
        y_train, y_test = y[train_index].copy(), y[test_index].copy()
        X_train , X_test = pca(X_train,X_test,kernel='rbf',n_components=min(7,len(X_train)//2)) ##PCA
        sm = SMOTE(random_state=0)
        X_train, y_train = sm.fit_resample(X_train, y_train)
        y_train = y_train.reshape(-1)
        y_train = np.array([row for row in y_train])
        if not run_only_cv:
            out = f_outpot[fold]
            get_stg_fun = out['get_stg_fun']
            score_gates = out['score_gates'][0:k]
            topk = out['topk'][0:k]
            col_name = out['col_name'][0:k]
            score_gates = out['score_gates'][0:k]
            history[f"fold_{fold}_score_gates"] = score_gates
            history[f"fold_{fold}_col_name"] = col_name
            history[f"fold_{fold}_fs_time"] = out['time']
        for clf in models:
            start = time.time()
            history[clf][fold] = {}
            estimators = [('clf', models[clf])]
            pipe = Pipeline(estimators)
            pipe.fit(X_train,y_train)
            stop_train = time.time()
            pred = pipe.predict_proba(X_test)
            stop_infer = time.time()
            history[clf][fold] = {}
            try:
                history[clf][fold] = {}
                history[clf][fold]['score'] = get_score(y_test,pred)
                history[clf][fold]['train_time'] = stop_train - start
                history[clf][fold]['infer_time'] =  stop_infer - stop_train
            except Exception as e:
                history[clf][fold]['train_time'] = stop_train - start
                history[clf][fold]['infer_time'] =  stop_infer - stop_train
                history[clf][fold]['Exception'] = e
                print(e)
                    
            history[clf]['y_score'].append(pred)
            history[clf]['y_val'].append(y_test)
            
    for clf in models:
        history[clf]['y_score'] = np.concatenate(history[clf]['y_score'])
        history[clf]['y_val'] = np.concatenate(history[clf]['y_val'])
        history[clf]['score']['cv_score'] = get_score(history[clf]['y_val'],history[clf]['y_score'])
        history[clf]['n_splits'] = n_splits
        history[clf]['name_cv'] = name_cv
        history[clf]['k'] = k
    return history
# history = train_kfold(Filtering_fun,Filtering_Algorithm,10,datsets_num)

In [5]:
def get_fs_per_fold(df,name,Filtering_Algorithm,train_features,y,train_index,test_index,k=100):
    start = time.time()
    outpot = {}
    X_train, X_test = train_features[train_index].copy(), train_features[test_index].copy()
    y_train, y_test = y[train_index].copy(), y[test_index].copy()
    get_stg_fun = Filtering[Filtering_Algorithm](datasets=name,out_path=f'temp/{Filtering_Algorithm}')
    score_gates = get_stg_fun(X_train,y_train,k=k)
    topk = score_gates.argsort()[::-1][0:k]
    col_name = np.array(df.drop('target',axis=1).columns)[topk]
    score_gates = score_gates[topk] 
    outpot['get_stg_fun'] = get_stg_fun
    outpot['score_gates'] = score_gates
    outpot['topk'] = topk
    outpot['col_name'] = col_name
    outpot['score_gates'] = score_gates
    outpot['time'] = time.time() - start
    return outpot

In [6]:
Filtering = {'STG':get_stg_class,
             'new_STG':get_stg_class_new,
            'f_classif':get_SelectFdr_class,
            'mrmr':get_mrmr_class,
            'reliefF':get_reliefF_class,
             'RFE_SVM':get_RFE_SVM_class,
            'FWDT':get_FWDT_class,
            'new_FWDT':get_FWDT_class_new,
            "ensemble":get_ensemble_class,
            "new_ensemble":get_ensemble_class_new}

In [7]:
def run_dataset(row):
    Filtering_Algorithm = row['Filtering Algorithm']
    name_d = row['Dataset Name']
    Learning_algorithm = row['Learning algorithm']
    n_jobs = 10
    topk=[row['Number of features selected (K)']]
    seedEverything(2022)
    start = time.time()
    ## Init
    os.makedirs(f"temp/aug_{Filtering_Algorithm}/",exist_ok=True)
    for datsets_num in range(63):
        df,name = get_specific_df(datsets_num)
        if name==name_d:
            break
    print(datsets_num)
    if os.path.exists(f'temp/aug_{Filtering_Algorithm}/{name}_history.pickle'):
        return
    PP = ColumnsPreprocessing(columns=name)
    Filtering_fun = Filtering[Filtering_Algorithm](datasets=name,out_path=f'temp/aug_{Filtering_Algorithm}')
    train_features,y = PP.transform(df)
    skf,n_splits,name_cv = get_cv_split(df)
    print("start")
    start = time.time()
    _ = Filtering_fun(train_features.values,y.values)
    end = time.time()-start
    print(end)
    run_only_cv = False if n_splits>10 else True
    split_fun = skf.split(train_features, y) if "Folds" in name_cv else skf.split(train_features)
    train_features = train_features.values
    y = y.values
    ## GridSearchCV
    # fun = Filtering[Filtering_Algorithm](datasets=name,out_path=f'temp/{Filtering_Algorithm}')
    seedEverything(2022)
    GridSearchCV_results = run_GridSearchCV(train_features,y,None,name,cv=3)
    ## Filtering_Algorithm
    outpot = None
    if run_only_cv:
        seedEverything(2022)
        outpot = Parallel(n_jobs=int(n_jobs))(delayed(get_fs_per_fold)(df,name,Filtering_Algorithm,train_features,y,train_index,test_index) for fold,(train_index, test_index) in enumerate(tqdm(split_fun))) 
    ## run trian
    seedEverything(2022)
    history = Parallel(n_jobs=int(1))(delayed(train_kfold)(Filtering_fun,Filtering_Algorithm,k,datsets_num,outpot,PP,end,Learning_algorithm,**GridSearchCV_results) for k in tqdm(topk))
    clear_output()
    print(time.time()-start)
    with open(f'temp/aug_{Filtering_Algorithm}/{name}_history.pickle', 'wb') as handle:
        pickle.dump(history, handle, protocol=4)

In [8]:
df = pd.read_csv("temp/results.csv")
algo = ['FWDT','f_classif','reliefF','STG','new_STG','new_FWDT','ensemble']
datasets = []
best = []
m = []
df_mini = df[df['Fold']=="CV Score"]
for i,mini in tqdm(df_mini.groupby("Dataset Name")):
    mini2 = mini[mini["Measure Type"] == 'AUC']
    best.append(mini2[mini2['Measure Value'] == mini2['Measure Value'].max()].reset_index(drop=True).iloc[0])
    datasets.append(i)

  0%|          | 0/60 [00:00<?, ?it/s]

In [9]:
# run_dataset(best[2])

In [ ]:
n_jobs = 64
log = Parallel(n_jobs=int(n_jobs))(delayed(run_dataset)(row) for row in tqdm(best))
clear_output()

  0%|          | 0/60 [00:00<?, ?it/s]

/home/kaor/.local/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  37   46   51   69   93  106  143  156  170  190  195  199  201  209
  217  220  228  232  238  240  242  251  254  260  267  275  281  291
  292  307  311  314  321  322  323  332  337  338  339  356  363  369
  374  395  401  411  430  438  449  450  456  462  469  482  506  517
  525  528  530  535  538  554  575  576  583  585  608  634  648  653
  678  682  715  724  729  773  778  780  797  814  815  816  820  822
  873  886  913  925  970 1046 1076 1132 1144 1170 1173 1175 1206 1216
 1218 1265 1278 1287 1293 1313 1318 1331 1333 1340 1356 1358 1371 1380
 1386 1398 1444 1447 1461 1507 1533 1546 1572 1579 1583 1624 1637 1640
 1699 1703 1730 1731 1742 1765 1777 1779 1784 1803 1809 1824 1829 1834
 1848 1902 1909 1920 1944 1954 1962 1968 1971 1992 1994 1995 1998 2017
 2055 2060 2093 2120 2127 2140 2160 2169 2183 2198 2238 2255 2274 2285
 2304 2321 2323 2325 233

3
start
0.026185989379882812



0it [00:00, ?it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [22] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/kaor/.local/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
0it [00:00, ?it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 6 members, which is less than n_splits=10.
  UserWarning,
10it [00:00, 666.99it/s]

4
start
0.0671391487121582


...

705.0989782810211
10
start
(6228,)
1107.8001601696014


  0%|          | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:1001: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning,
/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]

1it [00:00,  8.01it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,

2it [00:00,  8.46it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  UndefinedMetricWarning,

3it [00:00,  8.95it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No

888.0607838630676
1
start
(7129,)
1245.1134803295135


  0%|          | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:1001: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning,
/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]

1it [00:00,  6.66it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:1001: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  UndefinedMetricWarning,
/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]

2it [00:00,  7.05it/s]/home/kaor/.local/lib/python3.7/site-packages/sklearn/metrics/_ranking.py:992: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meani

934.6204779148102
